In [1]:
#=======================================================================================================================================
# 허깅페이스 Trainer를 이용하여 SOP+MLM 훈련시키기
#
# => load_dataset 으로 wiki 연속된 문장이 있는 말뭉치를 로딩하고, 이를 토크화 시키고, 
# 연속된 문장인지 아닌지 SOP 문장을 만들고, (Lable, sentence_order_label 필드 추가)
# 해당 문장 input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 pre-kowiki-20220620-2줄.txt 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817

# Albert config
# => albert-base-v2기준으로 생성
# => 참조: https://huggingface.co/albert-base-v2/blob/main/config.json 
#
# AlbertTokenizerFast
# => AlbertTokenizerFast 인데 한국어도 토큰화되도록 하려면, tokenizer_config.json에 masked_token-normalized:true 로 해줘야함.
# => 그렇지 않으면 한글 단어들은 모두 [unk] 으로 되어 버림 인데 
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AlbertTokenizer, AlbertTokenizerFast, BertTokenizerFast, AlbertConfig, AlbertForMaskedLM, AlbertForPreTraining

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# 훈련시킬 말뭉치
# => 한줄에 연속된 문장이 있어야함.
# -> .으로 구분된 한줄 문자이 아니라. 한줄에 .로구분된 여러문장이 이어진 문장이어야 함
# -> 예시:'제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
# => 사전 만들때 동일한 말뭉치 이용.
sep_string = ';$;'  # 말뭉치 2문장 구분자 => 문장A+[SET_STR]+문장B
input_corpus = '../../data11/ai_hub/tl1/tl1-2줄.txt' #'../../data11/ai_hub/tl1/tl1-2줄-test.txt' 
eval_corpus = '../../data11/ai_hub/vl1/vl1-2줄-eval.txt' #"../../data11/ai_hub/vl1/vl1-2줄-eval.txt"

# 기존 사전훈련된 모델
bispretrain = True       # 새로 pretrain 할꺼면 =True, 기존모델에 Further pretrain할꺼면 = False
bisscratch = False        # true = 완전 신규 scratch 모델 만듬, false= 기존 pretrain 모델을 좀더 pretrain 훈련시킴.

model_path = "bongsoo/albert-small-kor-v1"  

# 기존 사전 + 추가된 사전 파일
vocab_path="bongsoo/albert-small-kor-v1"  # bert vocab
#vocab_path="../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22M-vocab" ##albert vocab

# 출력
OUTPATH = '../../data11/model/albert/albert-small-kor-v1.1-checkout'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 128      # batch_size
token_max_len = 128   # token_seq_len
epoch = 4             # epoch
lr = 1e-4             # learning rate(기본:5e-5)
weigth_decay = 0.01   # weigth_decay(기본:0.0, bert: 0.01)
seed = 111
do_lower_case_ = True # 1=영어 대문자를 소문자로 변경, 0=대.소문자 구분
keep_acccents_ = False

# 기본 <unk>, <pad> 인데, 변경된 경우에는 아래 값을 변경해주면됨
unk_t ='[UNK]' #UNK 토큰 = <unk>
pad_t ='[PAD]' #PAD 토큰 = <pad>

############################################################################

device = GPU_info()
print(device)

#seed 설정
seed_everything(seed)

#logging 설정
logger =  mlogging(loggername="Albert-MLM-Trainer", logfilename="../../log/Albert-MLM-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/Albert-MLM-Trainer_2022-11-29.log


In [3]:
# tokeinzier 생성
# => AlbertTokenizer, AlbertTokenizerFast 둘중 사용하면됨.
# => AlbertTokenizerFast 인데 한국어도 토큰화되도록 하려면, tokenizer_config.json에 normalized:true 로 해줘야함.
# => 그렇지 않으면 한글 단어들은 모두 [unk] 으로 되어 버림

tokenizer = AlbertTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=do_lower_case_, keep_acccents=keep_acccents_, unk_token=unk_t, pad_token=pad_t)
#tokenizer = BertTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=do_lower_case_)

# fast 토크너나이즈인지 확인
print(f'*{vocab_path} is_fast:{tokenizer.is_fast}')
print('*tokenizer_len:{}, special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer), len(tokenizer.all_special_tokens), tokenizer.vocab_size))

if bispretrain == True:
    # 1.처음 부터 빈 껍데기 만들고 scratch 훈련 시키는 경우
    
    if bisscratch == True:
        # Albert 껍데기 만들기
        # => albert-base-v2기준으로 생성
        # => 참조: https://huggingface.co/albert-base-v2/blob/main/config.json
        config = AlbertConfig(    
            vocab_size=len(tokenizer), # default는 영어 기준이므로 내가 만든 vocab size에 맞게 수정해줘야 함
            hidden_size = 768,         # default는 4096인데, base는 768로 함
            num_attention_heads = 12,  # default는 64인데, base는 12로함
            intermediate_size = 3072,  # default는 16384인데 base는 3072로 함hidden_size * 4 = base는 3072임
            num_hidden_layers = 6      # default, base는 12인데, small버전으로 6으로 줄여봄
        )

        model = AlbertForPreTraining(config=config)
        
        print('*Scratch pretraining')
        
    else:
        # 2.기존 모델에 추가 pretraining 시키는 경우
        model = AlbertForPreTraining(model_path)
else:
    # 모델 로딩 further pre-training 
    model = AlbertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 

    #################################################################################
    # 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
    # 재설정하지 않으면, 다음과 같은 에러 발생함
    # CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
    #  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
    #
    #     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
    #     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
    #################################################################################
    model.resize_token_embeddings(len(tokenizer))

print(f'*num_param: {model.num_parameters()}')
model.to(device)

model

*../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22M-vocab is_fast:True
*tokenizer_len:30000, special_token_size: 5, *tokenizer.vocab_size: 30000
*num_param: 11813810


AlbertForPreTraining(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)

In [4]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
train_dataset = load_dataset("text", data_files=input_corpus) # text 로컬 파일 로딩

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

eval_dataset = load_dataset("text", data_files=eval_corpus) # text 로컬 파일 로딩

# eval_dataset 출력해봄
print(f"eval_dataset=======================================")
print(eval_dataset)
print(eval_dataset['train']['text'][0:3])


Using custom data configuration default-d9f8614839c15f8a
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/default-d9f8614839c15f8a/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29365626
    })
})
['정부, 무선국 검사제도 손본다;$;정부가 lte(롱텀에볼루션) 서비스 도입 등 바뀐 이동통신환경을 고려해 무선국 검사제도 규제개선을 추진한다.', '미래창조과학부는 효율적 전파관리 체계구축과 전파 이용자 편익증진을 위해 전파관리제도 개선 연구반이 도출한 무선국 검사제도 개선 방안을 시행한다고 16일 밝혔다.;$;개선안 주요 내용은 △표본검사대상 확대 및 표본비율 축소 △수시검사제도 도입 추진 △무선국 검사수수료 부과체계 합리화 등이다.', '미래부는 우선 이동통신사의 무선국 검사 부담 완화를 위해 현 준공검사 시 광중계기지국에 (이름) 시행되고 있던 표본검사를 준공검사 대상 전체 무선국으로 확대하고, 표본비율을 현 30%에서 향후 표본검사 시 불합격률을 고려해 단계적으로 20%까지 축소할 계획이다.;$;또 무선국 검사를 시행하는 비율을 축소할 경우 혼·간섭 없는 깨끗한 전파를 공급하기 위한 이동통신사업자들의 무선국 관리노력이 약화될 도덕적 해이가 우려돼 표본검사를 받지 않는 무선국에 대한 사후관리 제도인 수시검사 제도를 도입할 예정이다.']


Using custom data configuration default-b0e0740236a65728
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/default-b0e0740236a65728/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

eval_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7127
    })
})
["청호나이스, '먹는물·씻는물 안심' 정수기로 프리미엄시장 공략;$;청호나이스가 2가지 정수 시스템을 이용해 먹는 물과 씻는 물 모두 (이름)는 하이브리드 얼음정수기 ‘도도’를 출시했다.", '먹는 물에는 자사의 가장 뛰어난 필터 기술을 적용하고 식재료를 닦는 물에도 직수 필터 기술을 적용해 정수기 본질을 지킨 프리미엄 제품이라는 주장이다..;$;청호나이스는 환경오염과 생활 건강에 대한 소비자들 관심이 늘어나는 만큼 프리미엄 전략을 유지하며 직수 방식이나 다른 보급형 정수기를 쓰고 있는 가정을 고객으로 적극 끌어들일 계획이다.', '해외시장에도 활발히 나서 매년 국내외 제품 매출을 2배 수준으로 키운다는 포부다..;$;청호나이스는 9일 서울시 중구 소재 서울 웨스턴(이름)텔에서 하이브리드 얼음정수기 ‘도도’ 출시 발표회를 열었다..']


In [5]:
#=================================================================================================
# SOP 문장 만들기 
#
# => SOP는 주어진 한 쌍의 문장이 positive인지 negative인지 분류하는 이진 분류 문제이다.
# 문장 1: 그는 김치볶음밥을 요리했다.
# 문장 2: 맛있었다.
# =>주어진 한 쌍의 문장을 보면, 문장 2가 문장 1 다음에 온다는 것을 알 수 있다. 이때를 positive(0)라고 한다.

# 문장 1: 맛있었다.
# 문장 2: 그는 김치볶음밥을 요리했다.
# =>위 경우는 문장 순서가 바뀐 경우 이고 negetive(1)이다.
#=================================================================================================
import warnings
warnings.filterwarnings(action='ignore') 

import random
import copy
from tqdm.notebook import tqdm

def tokenizer_function_sop(examples):
     
    sentence_a = []
    sentence_b = []
    order_label = []

    count = 0
    for paragraph in examples['text']:
        count += 1
        # 하나의 문장을 읽어와서 .기준으로 나눈다.
        sentences = [sentence for sentence in paragraph.split(sep_string) if sentence != '']
        num_sentences = len(sentences)
         
         # . 기준으로 나눈 문장이 1이상이면..
        if num_sentences > 1:
            # 문장 a 시작번지는 랜덤하게, 해당 문장 이후로 지정
            start = random.randint(0, num_sentences-2)
            # 50/50 whether is IsNextSentence or NotNextSentence
            # 0.5 이상 랜덤값이면, 연속적인 문장으로 만듬
            if random.random() >= 0.5:
                # this is IsNextSentence
                sentence_a.append(sentences[start])
                sentence_b.append(sentences[start+1])
                order_label.append(0)  #label=0이면 연속적
            # 0.5 이하 랜덤값이면  순서를 바꿈
            else:
                # this is NotNextSentence
                sentence_a.append(sentences[start+1])
                sentence_b.append(sentences[start])
                order_label.append(1)  #label=1이면 비연속적
    
    # ** return_overflowing_tokenis = False로 해야, 긴 문장인 경우 잘리더라도 다시 이어서 문장을 만들지 않는다.
    # => 입력 문장은 10개인데, 긴문장이 포함된 경우 10개를 넘는 출력이 나옴
    #
    # ** albert 토크너나이즈 이용할때 아래 경고창 나옴. truncation='only_second' 하면 경고 몇개 안나오는데, 훈련할때 tensor size 오류 남.
    #Be aware, overflowing tokens are not returned for the setting you have chosen, 
    # i.e. sequence pairs with the 'longest_first' truncation strategy. 
    # So the returned list will always be empty even if some tokens have been removed.
    #
    result = tokenizer(sentence_a, sentence_b, max_length=token_max_len, padding=True, truncation=True, return_overflowing_tokens=False)
   
    # next_sentence_label next_label 복사(**deepcopy)해서 추가
    result['sentence_order_label'] = copy.deepcopy(order_label)
     
    return result

# 훈련 NSP 데이터셋은 빠른 기본 toeknzier_function 이용하여 만듬
%time train_dataset_fast = train_dataset.map(tokenizer_function_sop, batched=True)

# 평가 NSP 데이터셋은 빠른 기본 toeknzier_function 이용하여 만듬
%time eval_dataset_fast = eval_dataset.map(tokenizer_function_sop, batched=True)

  0%|          | 0/29366 [00:00<?, ?ba/s]

Loading cached processed dataset at /MOCOMSYS/.cache/huggingface/datasets/text/default-b0e0740236a65728/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4/cache-c126db7985bfdfd6.arrow


CPU times: user 6h 36min 3s, sys: 8h 12min 37s, total: 14h 48min 41s
Wall time: 1h 2min 6s
CPU times: user 11.5 ms, sys: 1.47 ms, total: 12.9 ms
Wall time: 12.2 ms


In [6]:
print(f"train_dataset_fast=======================================")
print(f'*train_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset_fast["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][0]))
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][1]))


print(f"eval_dataset_fast=======================================")
print(f'*eval_len:{len(eval_dataset_fast["train"])}, len:{len(eval_dataset_fast["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['train'][0:2])

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(eval_dataset_fast['train']['input_ids'][0]))
print(tokenizer.decode(eval_dataset_fast['train']['input_ids'][1]))


train_dataset_fast=======================================
*train_len:29365626, len:29365626
{'text': ['정부, 무선국 검사제도 손본다;$;정부가 lte(롱텀에볼루션) 서비스 도입 등 바뀐 이동통신환경을 고려해 무선국 검사제도 규제개선을 추진한다.', '미래창조과학부는 효율적 전파관리 체계구축과 전파 이용자 편익증진을 위해 전파관리제도 개선 연구반이 도출한 무선국 검사제도 개선 방안을 시행한다고 16일 밝혔다.;$;개선안 주요 내용은 △표본검사대상 확대 및 표본비율 축소 △수시검사제도 도입 추진 △무선국 검사수수료 부과체계 합리화 등이다.'], 'input_ids': [[5, 135, 3656, 3357, 1207, 424, 892, 447, 594, 1520, 670, 6, 135, 470, 5109, 342, 8461, 9568, 3518, 20207, 1247, 201, 678, 37, 7196, 1633, 2517, 1775, 17717, 5093, 820, 1663, 3357, 1207, 424, 892, 447, 751, 3452, 715, 5093, 253, 8727, 6676, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [5, 357, 1547, 439, 363, 1665, 306, 3517, 1520, 18062, 697, 717, 290, 117, 6900, 631, 3608, 2054, 306, 481, 442, 18062, 892, 447, 678, 253, 

In [7]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:1])

print(f"train_dataset_fast(MLM)=======================================")
print(train_dataset_fast)
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\norg===========================================\r\n')
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][0]))

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(mlm_train_sample['input_ids'][0]))


train_dataset_fast(MLM)=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'sentence_order_label'],
        num_rows: 29365626
    })
})
tensor([    5,   135,  3656,  3357,  1207,   424,   892,   447,   594,     7,
          670,     6,   135,   470,  5109,   342,  8461,  9568,  3518, 20207,
         1247,   201,   678, 24049,  7196,  1633,  2517,  1775, 17717,  5093,
          820,  1663,  3357,  1207,   424,   892,   447,     7,  3452,   715,
         5093,   253,  8727,  6676,     6,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3

In [8]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################
epochs = epoch          # epochs

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.02)            # 평가 스탭수(0.02)
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
#save_total_limit=2                                # 마지막 2개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    learning_rate=lr,                       # lr: 기본 5e-5
    weight_decay=weigth_decay,              # weigth_decay : 기본 = 0.0
    per_device_train_batch_size=batch_size,    # 배치 사이즈 
    save_strategy="epoch",                  # 저장 전략 (no, epoch, steps 기본=steps) 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    evaluation_strategy="steps",            # 평가 전략 (no, epoch, steps 기본=no)  
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

#  SOP 토크처리된 훈련 데이터셋
train_dataset_input = train_dataset_fast['train']
eval_dataset_input = eval_dataset_fast['train']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset_input,   # SOP 훈련 데이터셋
    eval_dataset=eval_dataset_input,     # SOP 평가 데이터셋
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


*total_optim_steps: 1835351, *eval_steps:36707, *logging_steps:36707, *save_steps:183535
*no_cuda: False


In [ ]:
# 훈련 시작
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForPreTraining.forward` and have been ignored: text. If text are not expected by `AlbertForPreTraining.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 29365626
  Num Epochs = 8
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1835352


Step,Training Loss,Validation Loss
36707,3.827300,2.864522
73414,2.619100,2.521642
110121,2.397400,2.376427
146828,2.281900,2.292671
183535,2.207900,2.265533
220242,2.156300,2.174724
256949,2.114400,2.143395
293656,2.081600,2.152235
330363,2.055800,2.091779
367070,2.033300,2.085398


The following columns in the evaluation set don't have a corresponding argument in `AlbertForPreTraining.forward` and have been ignored: text. If text are not expected by `AlbertForPreTraining.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `AlbertForPreTraining.forward` and have been ignored: text. If text are not expected by `AlbertForPreTraining.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `AlbertForPreTraining.forward` and have been ignored: text. If text are not expected by `AlbertForPreTraining.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding

In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../data11/model/albert/albert-aihub-SOP+MLM/'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')